In [16]:
import pandas as pd
from collections import Counter

In [6]:
data = pd.read_csv("operations.csv", parse_dates=[1,2]) 
# parse_dates permet de definir les colonnes de dates en Timestamp avec [colonne_de_date1,colonne_de_date2]
# Ajouter dayfirst=True si les dates sont au format jj-mm-aa

In [7]:
data.head()

,identifiant_transaction,date_operation,date_valeur,libelle,debit,credit,solde
0,242,2023-10-06,2023-10-06,FORFAIT COMPTE SUPERBANK XX XX XX XX,-1.92,NaN,2513.79
1,25,2023-10-06,2023-10-06,CARTE XX XX CHEZ LUC XX,-10.00,NaN,2529.29
2,69,2023-10-06,2023-10-06,PRELEVEMENT XX TELEPHONE XX XX,-13.58,NaN,2515.71
3,299,2023-10-05,2023-10-05,CARTE XX XX XX XX XX XX,-10.64,NaN,2539.29
4,45,2023-10-05,2023-10-05,CARTE XX XX XX XX,-4.80,NaN,2549.93


In [15]:
col_names = ['identifiant_transaction','date_operation','date_valeur','libelle','debit','credit','solde']

assert len(data.columns) == len(col_names)
for i in range(len(data.columns)):
    assert data.columns[i] == col_names[i]

In [21]:
def most_common_words(labels,n_word_to_print):
    '''Fonction pour afficher les valeurs les plus frequentes d'une colonne et leur compte
    Input:
    - labels: Pandas Colonne 
    - n_word_to_print: int du nombre de mot à afficher
    Output: None mais print les elements '''
    
    words = []
    for lab in labels:
        words += lab.split(" ")
    counter = Counter(words) #Counter de la librairie collections
    for word in counter.most_common(n_word_to_print):
        print(word)

most_common_words(data['libelle'],10)

('XX', 1428)
('CARTE', 247)
('VIREMENT', 29)
('LES', 20)
('ANCIENS', 20)
('ROBINSON', 20)
('CHEZ', 16)
('LUC', 16)
('LA', 16)
("L'EPICERIE", 16)


In [22]:
# Creation des TYPES et des CATEGORIES des opérations du compte en banque

CATEGS = {
    'LOYER': 'LOYER',
    'FORFAIT COMPTE SUPERBANK': 'COTISATION BANCAIRE',
    'LES ANCIENS ROBINSON': 'COURSES',
    "L'EPICERIE DENBAS": 'COURSES',
    'TELEPHONE': 'FACTURE TELEPHONE',
    'LA CCNCF': 'TRANSPORT',
    'CHEZ LUC': 'RESTAURANT',
    'RAPT': 'TRANSPORT',
    'TOUPTIPRI': 'COURSES',
    'LA LOUVE': 'COURSES',
    'VELOC': 'TRANSPORT'
}

TYPES = {
    'CARTE': 'CARTE',
    'VIR': 'VIREMENT',
    'VIREMENT': 'VIREMENT',
    'RETRAIT': 'RETRAIT',
    'PRLV': 'PRELEVEMENT',
    'DON': 'DON',
}

In [23]:
LAST_BALANCE = 2400 # Solde du compte APRES la dernière opération en date
EXPENSES = [50,150] # Bornes des catégories de dépense : [petite,grosse]
WEEKEND = ["Saturday","Sunday"] # Jours non travaillés

In [24]:
## Controle des colonnes ##
for c in ['date_operation','libelle','debit','credit']:
    if c not in data.columns:
        if (c in ['debit','credit'] and 'montant' not in data.columns) or \
        (c not in ['debit','credit']):
            msg = "Il vous manque la colonne '{}'. Attention aux majuscules "
            msg += "et minuscules dans le nom des colonnes!"
            raise Exception(msg.format(c))

## Suppression des colonnes innutiles ##
for c in data.columns:
    if c not in ['date_operation','libelle','debit','credit','montant']:
        del data[c]

## Ajout de la colonne 'montant' si besoin ##
if 'montant' not in data.columns:
    data["debit"] = data["debit"].fillna(0)
    data["credit"] = data["credit"].fillna(0)
    data["montant"] = data["debit"] + data["credit"]
    del data["credit"], data["debit"]

## creation de la variable 'solde_avt_ope' ##
data = data.sort_values("date_operation")
amount = data["montant"]
balance = amount.cumsum()
balance = list(balance.values)
last_val = balance[-1]
balance = [0] + balance[:-1]
balance = balance - last_val + LAST_BALANCE
data["solde_avt_ope"] = balance

In [25]:
def detect_words(values, dictionary):
    '''Fonction qui assigne les operations a une CATEGORIE et a un TYPE'''
    
    result = []
    for lib in values:
        operation_type = "AUTRE"
        for word, val in dictionary.items():
            if word in lib:
                operation_type = val
        result.append(operation_type)
    return result

data["categ"] = detect_words(data["libelle"], CATEGS)
data["type"] = detect_words(data["libelle"], TYPES)

In [26]:
def expense_slice(value):
    '''Creation des variables "tranche_depense" et "sens" '''
    
    value = -value # Les dépenses sont des nombres négatifs
    if value < 0:
        return "(pas une dépense)"
    elif value < EXPENSES[0]:
        return "petite"
    elif value < EXPENSES[1]:
        return "moyenne"
    else:
        return "grosse"
    
data["tranche_depense"] = data["montant"].map(expense_slice)
data["sens"] = ["credit" if m > 0 else "debit" for m in data["montant"]]

In [30]:
## Creation des autres variables: jour, mois, annee, week-end, semaine,...
data["annee"] = data["date_operation"].map(lambda d: d.year)
data["mois"] = data["date_operation"].map(lambda d: d.month)
data["jour"] = data["date_operation"].map(lambda d: d.day)
data["jour_sem"] = data["date_operation"].map(lambda d: d.weekday_name)
data["jour_sem_num"] = data["date_operation"].map(lambda d: d.weekday()+1)
data["weekend"] = data["jour_sem"].isin(WEEKEND)
data["quart_mois"] = [int((jour-1)*4/31)+1 for jour in data["jour"]]

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: `weekday_name` is deprecated and will be removed in a future version. Use `day_name` instead
  """


In [31]:
## Enregistrement au format CSV ##
data.to_csv("operations_enrichies.csv",index=False)

In [32]:
data.head()

,date_operation,libelle,montant,solde_avt_ope,categ,type,tranche_depense,sens,annee,mois,jour,jour_sem,jour_sem_num,weekend,quart_mois
307,2023-03-31,DON XX XX XX XX XX XX XX,-1.44,1515.25,AUTRE,DON,petite,debit,2023,3,31,Friday,5,False,4
302,2023-04-03,CARTE XX XX RAPT XX,-24.00,1513.81,TRANSPORT,CARTE,petite,debit,2023,4,3,Monday,1,False,1
303,2023-04-03,CARTE XX XX RAPT XX,-73.00,1489.81,TRANSPORT,CARTE,moyenne,debit,2023,4,3,Monday,1,False,1
306,2023-04-03,CARTE XX XX XX XX,-14.39,1416.81,AUTRE,CARTE,petite,debit,2023,4,3,Monday,1,False,1
305,2023-04-03,VIREMENT XX XX XX XX XX XX,4.80,1402.42,AUTRE,VIREMENT,(pas une dépense),credit,2023,4,3,Monday,1,False,1
